yatb - yet another test bench

Todo: Create a common test tool for all models. It should use a translator as input, and output bleu score for the test-set of choice.

In [5]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from utils.preprocess import fields_for_rule_text
from torchtext.legacy.data import Field

In [6]:
SRC, TRG = fields_for_rule_text()
fields = {'src-rule': ('src', SRC), 'trg-rule': ('trg', TRG)}
fields = {'src-rule': ('src', Field()), 'trg-rule': ('trg', Field())}
data = TestSets.load(fields)

In [7]:
print(vars(data.examples[0]))

{'src': ['Whenever', 'another', 'artifact', 'or', 'creature', 'you', 'control', 'is', 'put', 'into', 'a', 'graveyard', 'from', 'the', 'battlefield,', 'put', 'an', 'oil', 'counter', 'on', 'Vat', 'of', 'Rebirth.', '{2}{B},', '{T},', 'Remove', 'four', 'oil', 'counters', 'from', 'Vat', 'of', 'Rebirth:', 'Return', 'target', 'creature', 'card', 'from', 'your', 'graveyard', 'to', 'the', 'battlefield.', 'Activate', 'only', 'as', 'a', 'sorcery.'], 'trg': ['每当另一个由你操控的神器或生物从战场进入坟墓场时，在再生缸上放置一个烁油指示物。', '{2}{B}，{T}，从再生缸上移去四个烁油指示物：将目标生物牌从你的坟墓场移回战场。只能于法术时机起动。']}


In [6]:
from models.card_name_detector.definition import TrainedDetector

D = TrainedDetector()
D.annotate('when xbp, NKU\'s hope enters the battlefield, search your library for a card named ddw, xbp\'s minion, put it into your hand.',
           threshold=1e-3)

path: d:\ddw\school\大三下\语音信息处理技术\期末作业\code\mtg-cards-translation\models\card_name_detector
['when', 'xbp', ',', 'nku', "'s", 'hope', 'enters', 'the', 'battlefield', ',', 'search', 'your', 'library', 'for', 'a', 'card', 'named', 'ddw', ',', 'xbp', "'s", 'minion', ',', 'put', 'it', 'into', 'your', 'hand', '.']


" when <xbp , nku 's hope> enters the battlefield , search your library for a card named <ddw , xbp 's minion> , put it into your hand ."